In [1]:
import cv2 as cv
from cv2 import aruco
import numpy as np

In [2]:
marker_dict=aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
MARKER_SIZE=400 #pixels

In [24]:
for id in range(20):
#     marker_img=aruco.drawMarker(marker_dict,id,MARKER_SIZE)
    marker_img=aruco.generateImageMarker(marker_dict,id,MARKER_SIZE)

    cv.imshow('marker',marker_img)
#     cv.imwrite(f'markers/marker_{id}.png',marker_img)
    cv.waitKey(0)
    break

In [3]:
## detecting aruco markers
param_marker=aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(marker_dict, param_marker)
cap=cv.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if not ret:
        break
    gray_frame=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    marker_corners,marker_id,reject=detector.detectMarkers(gray_frame)
    if marker_id is not None:
        for ids,corner in zip(marker_id,marker_corners):
            cv.polylines(frame,[corner.astype(np.int32)],True,(0,255,255),4,cv.LINE_AA)
            cv.putText(frame,f'id:{ids}',(50,50),cv.FONT_HERSHEY_COMPLEX,1,(0,255,255),2)
#             print(ids,corner)
    cv.imshow('video',frame)
    key=cv.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv.destroyAllWindows()

In [17]:
#building image augumentation
def image_augumentation(frame,src_image,target_points):
    src_h,src_w=src_image.shape[:2]
    frame_h,frame_w=frame.shape[:2]
    mask=np.zeros((frame_h,frame_w),dtype=np.uint8)
    src_points=np.array([[0,0],[src_w,0],[src_w,src_h],[0,src_w]])
    H,_=cv.findHomography(src_points,target_points)
    warp_image=cv.warpPerspective(src_image,H,(frame_w,frame_h))
    cv.imshow('warp',warp_image)
    cv.fillConvexPoly(mask,target_points,255)
    results=cv.bitwise_and(warp_image,warp_image,frame,mask=mask)

In [18]:
## detecting aruco markers
param_marker=aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(marker_dict, param_marker)
cap=cv.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if not ret:
        break
    gray_frame=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    marker_corners,marker_id,reject=detector.detectMarkers(gray_frame)
    aug_img=cv.imread('test_ronaldo.jpg')
    if marker_id is not None:
        for ids,corner in zip(marker_id,marker_corners):
            corner=corner.reshape(4,2)
            corner=corner.astype(int)
            image_augumentation(frame,aug_img,corner)
    cv.imshow('video',frame)
    key=cv.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv.destroyAllWindows()